<a href="https://colab.research.google.com/github/Tesnime/stagePFE/blob/main/AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install fastapi uvicorn pyngrok

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.7/94.7 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.3/73.3 kB 2.9 MB/s eta 0:00:00


In [ ]:
!pip install nltk
!pip install transformers
!pip install torch
!pip install sentencepiece

In [ ]:
!pip install scikit-surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 3.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp310-cp310-linux_x86_64.whl size=2357292 sha256=bd5efda713983acf6a09f08823e827fbc6ff3c40dcbdc559696a652a556b3597
  Stored in directory: /root/.cache/pip/wheels/4b/3f/df/6acbf0a40397d9bf3ff97f582cc22fb9ce66adde75bc71fd54
Successfully built scikit-surprise


In [ ]:
!pip install pyngrok

In [ ]:
!ngrok authtoken 2mM5nTH7jR1D2Wh35BUlojVlNmG_5P5gEuRsZAp9qxsuRBNz1

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
!pip install langdetect
!pip install phonenumbers
!pip install pdfplumber spacy
!python -m spacy download fr_core_news_sm
!pip install PyMuPDF spacy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 11.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993221 sha256=59759330cdafb1658e3fafba66045b301a78be081bf8901fdac4b5748ab83245
  Stored in directory: /root/.cache/pip/wheels/95/03/7d/59ea870c70ce4e5a370638b5462a7711ab78fba2f655d05106
Successfully built langdetect
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 39.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 40.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.3/16.3 MB 66.8 MB/s eta 0:00:00
✔ Download and installation su

In [ ]:
!pip install python-multipart

In [ ]:
import pandas as pd
from fastapi import FastAPI, UploadFile, File, HTTPException
from pydantic import BaseModel
from typing import List, Optional
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression
from collections import Counter
from pyngrok import ngrok
import nest_asyncio
import uvicorn
from transformers import AutoModelForSequenceClassification, AutoTokenizer, pipeline
import fitz  # PyMuPDF
import os
import shutil
import spacy
import re

# Load the spaCy model
nlp = spacy.load("fr_core_news_sm")
# Apply nest_asyncio patch
nest_asyncio.apply()

# Load the data
formateur_df = pd.read_csv('/content/drive/MyDrive/datasets/formateur_com_tech (1).csv')
demande_df = pd.read_csv('/content/drive/MyDrive/datasets/demande (1).csv')
evaluation_apprenant_df = pd.read_csv('/content/drive/MyDrive/datasets/evaluation_apprenant (1).csv')
difficulties_df = pd.read_csv('/content/drive/MyDrive/datasets/interactions_difficultes.csv')
interests_df = pd.read_csv('/content/drive/MyDrive/datasets/interactions_centre_interet.csv')
demande_theme = pd.read_csv('/content/drive/MyDrive/datasets/demande_theme (2).csv')

model_name = "cardiffnlp/twitter-xlm-roberta-base-sentiment"
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
classifier = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

# Merge the two dataframes on 'interactions_id'
merged_df = pd.merge(difficulties_df, interests_df, on='interactions_id', how='outer')

# Group the merged dataframe by 'interactions_id' and aggregate the difficulties and interests into lists
grouped_df = merged_df.groupby('interactions_id').agg({
    'difficultes': lambda x: list(x.unique()),
    'centre_interet': lambda x: list(x.unique())
}).reset_index()

# Count occurrences of each theme from both difficulties and interests
theme_counts = Counter()
for index, row in grouped_df.iterrows():
    theme_counts.update(row['difficultes'])
    theme_counts.update(row['centre_interet'])

# Convert counts to a DataFrame for easy viewing and sort by count in descending order
theme_counts_df = pd.DataFrame(theme_counts.items(), columns=['Theme', 'Count']).sort_values(by='Count', ascending=False)

# Replace missing values in the 'star' column with the mean rating
mean_star = evaluation_apprenant_df['star'].mean()
evaluation_apprenant_df['star'].fillna(mean_star, inplace=True)

# Merge evaluation data with demande to access formateur_id
evaluation_with_formateur = pd.merge(
    evaluation_apprenant_df,
    demande_df[['id', 'formateur_id']],
    left_on='id',
    right_on='id'
)

# Group by formateur_id to get the average rating for each trainer
grouped_evaluation = evaluation_with_formateur.groupby('formateur_id')['star'].mean().reset_index()

# Prepare data for RandomForestRegressor
X_formateur = formateur_df[['com_tech']]
y_formateur = grouped_evaluation.set_index('formateur_id').reindex(formateur_df['formateur_id'])['star'].dropna()

# Remove rows where y_formateur is NaN
X_formateur_encoded = pd.get_dummies(X_formateur).reindex(index=y_formateur.index, fill_value=0)

# Train the model
model_formateur = RandomForestRegressor(random_state=42)
model_formateur.fit(X_formateur_encoded, y_formateur)

# Prepare for regression-based theme recommendations
demande_df.rename(columns={'id': 'demande_id'}, inplace=True)
demande_with_scores = pd.merge(demande_df, evaluation_apprenant_df, on='demande_id', how='left')


def get_best_formateur():
    print(demande_df.columns)
    # Group by demande_id and calculate the mean star rating
    formateur_ratings = evaluation_apprenant_df.groupby('demande_id')['star'].mean().reset_index()
    formateur_ratings.columns = ['demande_id', 'average_star']

    print("Formateur Ratings:\n", formateur_ratings)  # Debug print

    # Merge the ratings with the demande dataframe to link it to formateurs
    formateur_performance = pd.merge(demande_df, formateur_ratings, left_on='demande_id', right_on='demande_id', how='inner')

    if formateur_performance.empty:
        print("Formateur Performance is empty")  # Debug print
        return None

    print("Formateur Performance:\n", formateur_performance)  # Debug print

    # Merge formateur_performance with formateurComp to get formateurs' competencies
    best_formateurs = pd.merge(formateur_performance, formateur_df, on='formateur_id', how='inner')

    if best_formateurs.empty:
        print("Best Formateurs is empty")  # Debug print
        return None

    print("Best Formateurs:\n", best_formateurs)  # Debug print

    best_formateurs = best_formateurs.sort_values(by='average_star', ascending=False)

    best_formateur_id = int(best_formateurs.iloc[0]['formateur_id'])
    return best_formateur_id




# Function to prepare data for regression
def prepare_data(formateur_id):
    competencies = formateur_df[formateur_df['formateur_id'] == formateur_id]['com_tech'].values
    if competencies.size == 0:
        return None

    relevant_demands = demande_with_scores[demande_with_scores['formateur_id'] == formateur_id]
    regression_data = relevant_demands[['demande_id', 'theme', 'star']].copy()

    mean_star = regression_data['star'].mean()
    regression_data['star'] = regression_data['star'].fillna(mean_star)

    if regression_data.empty:
        return None

    regression_data['competency'] = competencies[0]
    return regression_data

# Function to recommend themes using regression
def recommend_themes_for_formateur(formateur_id):
    regression_data = prepare_data(formateur_id)
    if regression_data is None:
        return []

    y = regression_data['star']

    theme_encoder = OneHotEncoder(sparse_output=False)
    X_theme_encoded = theme_encoder.fit_transform(regression_data[['theme']])

    competency_encoder = OneHotEncoder(sparse_output=False)
    X_comp_encoded = competency_encoder.fit_transform(regression_data[['competency']])

    X_final = pd.DataFrame(X_theme_encoded, columns=theme_encoder.get_feature_names_out(['theme']))
    X_comp_final = pd.DataFrame(X_comp_encoded, columns=competency_encoder.get_feature_names_out(['competency']))

    X_final = pd.concat([X_final, X_comp_final], axis=1)

    model = LinearRegression()
    model.fit(X_final, y)

    predictions = model.predict(X_final)
    regression_data['predicted_score'] = predictions

    recommendations = pd.merge(regression_data, demande_theme, on='demande_id', how='left')
    recommended_themes = recommendations.sort_values(by='predicted_score', ascending=False)

    top_themes = recommended_themes[['theme_y']].drop_duplicates().head(7)
    return top_themes['theme_y'].tolist()

# Prediction function for suitable trainers by theme
def predict_formateur_by_theme(themes: List[str]) -> pd.DataFrame:
    themes_set = set(theme.lower().strip() for theme in themes)

    def has_matching_comp(com_list):
        return any(comp.lower().strip() in themes_set for comp in com_list.split(','))

    # Filter formateurs to find matching trainers
    filtered_df = formateur_df[formateur_df['com_tech'].apply(has_matching_comp)]

    if filtered_df.empty:
        return pd.DataFrame(columns=['formateur_id', 'predicted_star'])

    # Prepare data for prediction
    X_predict = pd.get_dummies(filtered_df[['com_tech']])
    X_predict = X_predict.reindex(columns=X_formateur_encoded.columns, fill_value=0)

    # Make predictions
    filtered_df['predicted_star'] = model_formateur.predict(X_predict)

    # Return the top 4 trainers based on predicted_star
    top_trainers = filtered_df.nlargest(4, 'predicted_star')

    return top_trainers[['formateur_id', 'predicted_star']]

# Initialize FastAPI app
app = FastAPI()

# Pydantic models for input requests
class TrainerRequest(BaseModel):
    themes: List[str]

class FormateurIDRequest(BaseModel):
    formateur_id: int

class CommentRequest(BaseModel):
    comment: str  # Single comment

# Function to check toxicity of a comment
def check_comment(comment: str) -> bool:
    results = classifier(comment)
    for result in results:
        if result['label'] == 'negative' and result['score'] > 0.5:  # Toxicity threshold
            return False  # Inappropriate comment
    return True  # Acceptable comment

# class CVData(BaseModel):
#     name: Optional[str]
#     position: Optional[str]
#     profile: Optional[str]
#     experience: List[str]
#     education: List[str]
#     certifications: Optional[str]
#     contact: Optional[str]
#     skills: List[str]
#     languages: Optional[str]
#     interests: Optional[str]
#     email: Optional[str]
#     address: Optional[str]

# def extract_cv_data(pdf_path):
#     # Your existing extraction logic here
#     doc = fitz.open(pdf_path)
#     text = ""
#     for page_num in range(len(doc)):
#         page = doc.load_page(page_num)
#         text += page.get_text()

#     data = {
#         'name': extract_info(text, 'Nom Prénom', 'Intitulé du poste'),
#         'position': extract_info(text, 'Intitulé du poste', 'Profil Professionnel'),
#         'profile': clean_profile(extract_info(text, 'Profil Professionnel', 'Expérience Professionnelle')),
#         'experience': split_experience(extract_info(text, 'Expérience Professionnelle', 'Formation')),
#         'education': split_education(extract_info(text, 'Formation', 'Certifications')),
#         'certifications': extract_info(text, 'Certifications', 'Contact'),
#         'contact': clean_contact(extract_info(text, 'Contact', 'Compétences')),
#         'skills': split_skills(extract_info(text, 'Compétences', 'Langues')),
#         'languages': extract_info(text, 'Langues', 'Centres d\'intérêt'),
#         'interests': extract_info(text, 'Centres d\'intérêt', 'Certificat d\'Aptitude'),
#     }

#     email, address = extract_email_and_address(text)
#     data['email'] = email
#     data['address'] = address

#     return data

# @app.post("/upload_cv", response_model=CVData)
# async def upload_cv(file: UploadFile = File(...)):
#     # Save the uploaded file to a temporary location
#     pdf_path = f"/tmp/{file.filename}"

#     with open(pdf_path, "wb") as buffer:
#         buffer.write(await file.read())

#     try:
#         # Extract CV data from the uploaded PDF
#         cv_data = extract_cv_data(pdf_path)
#     finally:
#         # Clean up the temporary file
#         os.remove(pdf_path)

#     return cv_data



class CVData(BaseModel):
    name: Optional[str]
    position: Optional[str]
    profile: Optional[str]
    experience: List[str]
    education: List[str]
    certifications: Optional[str]
    contact: Optional[str]
    skills: List[str]
    languages: Optional[str]
    interests: Optional[str]
    email: Optional[str]
    address: Optional[str]
    phone: Optional[str]


def extract_phone_using_nlp(text):
    """Extract phone numbers using NLP techniques."""
    doc = nlp(text)
    phone_numbers = []

    for token in doc:
        # Simple heuristic: check for tokens that might represent phone numbers
        if token.like_num and (len(token.text) >= 7 and len(token.text) <= 15):
            # Assume tokens with certain lengths are part of a phone number
            phone_numbers.append(token.text)

    # Join the found tokens to form a potential phone number
    if phone_numbers:
        return " ".join(phone_numbers)
    return None

def extract_info(text, start_marker, end_marker):
    """Extract information between two markers in the text."""
    start_index = text.find(start_marker)
    end_index = text.find(end_marker, start_index)
    if start_index != -1 and end_index != -1:
        return text[start_index + len(start_marker):end_index].strip()
    return None

def extract_position_using_nlp(text):
    """Extract position title using NLP."""
    doc = nlp(text)
    position = None

    # Simple heuristic: looking for phrases that typically indicate a position title
    for token in doc:
        if token.dep_ == "nsubj" and token.pos_ == "PROPN":
            position = token.text
            break

    # Vous pouvez affiner cette logique pour mieux détecter les intitulés de poste
    return position

def clean_profile(profile_text):
    """Clean and format the profile section."""
    if profile_text:
        return profile_text.strip()
    return None

def split_experience(experience_text):
    """Split the experience section into a list of experiences."""
    if experience_text:
        return [exp.strip() for exp in experience_text.split("\n") if exp.strip()]
    return []

def split_education(education_text):
    """Split the education section into a list of education entries."""
    if education_text:
        return [edu.strip() for edu in education_text.split("\n") if edu.strip()]
    return []

def clean_contact(contact_text):
    """Clean and format the contact section."""
    if contact_text:
        return contact_text.strip()
    return None

def split_skills(skills_text):
    """Split the skills section into a list of skills."""
    if skills_text:
        return [skill.strip() for skill in skills_text.split(",") if skill.strip()]
    return []

def extract_email_and_address(text):
    """Uses spaCy to extract email and address from the text."""
    doc = nlp(text)
    email = None
    address = ""

    # Find email
    for token in doc:
        if token.like_email:
            email = token.text
            break

    # Find address by looking for keywords like street numbers, names, and postal codes
    for ent in doc.ents:
        if ent.label_ == "GPE" or ent.label_ == "LOC" or ent.label_ == "FAC" or ent.label_ == "ORG" or ent.label_ == "ADDRESS":
            if "Rue" in ent.text or "Avenue" in ent.text or any(char.isdigit() for char in ent.text):
                address += ent.text + " "

    # Cleaning up the address to ensure it is coherent
    address = address.strip()

    return email, address

def extract_cv_data(pdf_path):
    # Open the PDF and extract text
    doc = fitz.open(pdf_path)
    text = ""
    for page_num in range(len(doc)):
        page = doc.load_page(page_num)
        text += page.get_text()

    # Extract data based on sections
    data = {
        'name': extract_info(text, 'Nom Prénom', 'Intitulé du poste'),
        'position': extract_position_using_nlp(text),
        'profile': clean_profile(extract_info(text, 'Profil Professionnel', 'Expérience Professionnelle')),
        'experience': split_experience(extract_info(text, 'Expérience Professionnelle', 'Formation')),
        'education': split_education(extract_info(text, 'Formation', 'Certifications')),
        'certifications': extract_info(text, 'Certifications', 'Contact'),
        'contact': clean_contact(extract_info(text, 'Contact', 'Compétences')),
        'skills': split_skills(extract_info(text, 'Compétences', 'Langues')),
        'languages': extract_info(text, 'Langues', 'Centres d\'intérêt'),
        'interests': extract_info(text, 'Centres d\'intérêt', 'Certificat d\'Aptitude'),
    }

    # Extract email and address separately
    email, address = extract_email_and_address(text)
    phone = extract_position_using_nlp(text)
    data['email'] = email
    data['address'] = address
    data['phone'] = phone

    return CVData(**data)

@app.post("/upload_cv", response_model=CVData)
async def upload_cv(file: UploadFile = File(...)):
    # Save the uploaded file to a temporary location
    pdf_path = f"/tmp/{file.filename}"

    try:
        # Open the file in write-binary mode
        with open(pdf_path, "wb") as buffer:
            shutil.copyfileobj(file.file, buffer)

        # Extract CV data from the uploaded PDF
        cv_data = extract_cv_data(pdf_path)

    except Exception as e:
        raise HTTPException(status_code=500, detail=f"An error occurred while processing the file: {e}")

    finally:
        # Clean up the temporary file
        if os.path.exists(pdf_path):
            os.remove(pdf_path)

    return cv_data








# @app.post("/upload_cv")
# async def upload_cv(file: UploadFile = File(...)):
#     content = await file.read()  # Read the file content
#     return {"filename": file.filename, "content_length": len(content)}

# FastAPI route to check a single comment for toxicity
@app.post("/check-comment/")
def check_single_comment(request: CommentRequest):
    comment_status = check_comment(request.comment)
    return comment_status

# Define the FastAPI route for finding suitable trainers
@app.post("/suitable-trainers/")
def find_suitable_trainers(request: TrainerRequest):
    trainers = predict_formateur_by_theme(request.themes)
    return {"suitable_trainers": trainers.to_dict(orient='records')}


# Define the FastAPI route for recommending themes based on a formateur
@app.post("/recommend-themes-by-formateur/")
def recommend_themes(formateur_id: int):
    recommended_themes = recommend_themes_for_formateur(formateur_id)
    return {"recommended_themes": recommended_themes}

# Define the FastAPI route for recommending popular themes
@app.get("/recommend-themes/")
def recommend_themes():
    recommended_themes = theme_counts_df.nlargest(5, 'Count')['Theme'].tolist()
    return {"recommended_themes": recommended_themes}


@app.get("/best-formateur/")
def get_best_formateur_endpoint():
    best_formateur_id = get_best_formateur()
    if best_formateur_id:
        return {"best_formateur_id": best_formateur_id}
    else:
        return {"message": "No formateur data available"}
# Create a public URL using ngrok
public_url = ngrok.connect(8000)
print(f"FastAPI app is running on {public_url}")

# Run the FastAPI app with Uvicorn
if __name__ == "__main__":
    uvicorn.run(app, host="0.0.0.0", port=8000)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/841 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
<ipython-input-8-ae178f3a4d82>:57: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  evaluation_apprenant_df['star'].fillna(mean_star, inplace=Tru

FastAPI app is running on NgrokTunnel: "https://7f37-34-91-99-153.ngrok-free.app" -> "http://localhost:8000"
INFO:     197.0.37.248:0 - "POST /recommend-themes-by-formateur/?formateur_id=723 HTTP/1.1" 200 OK
INFO:     197.0.37.248:0 - "POST /check-comment/ HTTP/1.1" 200 OK
INFO:     197.0.37.248:0 - "POST /check-comment/ HTTP/1.1" 200 OK
Index(['demande_id', 'cv', 'nbr_session', 'theme', 'formateur_id', 'status',
       'titre', 'date', 'raison', 'description', 'doc', 'code'],
      dtype='object')
Formateur Ratings:
 INFO:     197.0.37.248:0 - "GET /recommend-themes/ HTTP/1.1" 200 OK
   demande_id  average_star
0           5      2.666667
1          33      4.000000
Formateur Performance:
    demande_id  cv  nbr_session  theme  formateur_id    status  \
0           5 NaN            4    NaN           4.0  terminer   
1          33 NaN            3    NaN         721.0  terminer   

                  titre                        date raison  \
0      titre de demande                   